# 03 Production Training - Full Dataset (11,979 samples)

**Features**:
- ✅ Uses ALL real TOI data (11,979 samples)
- ✅ Checkpoint system - resume if interrupted
- ✅ Batch processing with progress tracking
- ✅ Parallel processing for speed
- ✅ Automatic retry on failures
- ✅ Memory efficient

**Workflow**: Load → Extract Features (with checkpoints) → Train → Save

**Estimated Time**: 4-8 hours for full dataset (depends on network)

## Cell 1: Installation & Setup

In [ ]:
# Installation (Colab only)
import sys
if 'google.colab' in sys.modules:
    print("📦 Installing dependencies...")
    !pip install -q numpy==1.26.4 'scipy<1.13' lightkurve pandas scikit-learn xgboost joblib tqdm
    !pip install -q transitleastsquares pyarrow
    print("✅ Installation complete")
    print("⚠️ If you see errors, restart runtime and skip to Cell 2")
else:
    print("✅ Running locally")

## Cell 2: Imports & Configuration

In [ ]:
# Standard library
import os
import sys
import json
import warnings
from pathlib import Path
from typing import Dict, List, Optional
from datetime import datetime
import time

# Scientific computing
import numpy as np
import pandas as pd
from tqdm import tqdm

# Machine learning
from sklearn.model_selection import StratifiedGroupKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score, roc_auc_score, classification_report
import xgboost as xgb
import joblib

# Astronomy
import lightkurve as lk
from astropy.timeseries import BoxLeastSquares

# Suppress warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

print("✅ All imports successful")
print(f"   NumPy: {np.__version__}")
print(f"   Pandas: {pd.__version__}")
print(f"   XGBoost: {xgb.__version__}")
print(f"   Lightkurve: {lk.__version__}")

## Cell 3: Setup Paths & Clone Repository

In [ ]:
# Detect environment
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌍 Running in Google Colab")
    
    # Mount Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    
    # Clone repository if needed
    REPO_DIR = Path('/content/exoplanet-starter')
    if not REPO_DIR.exists():
        print("📥 Cloning repository...")
        !git clone https://github.com/exoplanet-spaceapps/exoplanet-starter.git /content/exoplanet-starter
        print("✅ Repository cloned")
    else:
        print("✅ Repository already exists")
    
    os.chdir(str(REPO_DIR))
    
    # Paths
    BASE_DIR = REPO_DIR
    CHECKPOINT_DIR = Path('/content/drive/MyDrive/spaceapps-checkpoints')
    MODEL_DIR = Path('/content/drive/MyDrive/spaceapps-models')
    
else:
    print("💻 Running locally")
    BASE_DIR = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
    CHECKPOINT_DIR = BASE_DIR / 'checkpoints'
    MODEL_DIR = BASE_DIR / 'models'

DATA_DIR = BASE_DIR / 'data'
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✅ Paths configured:")
print(f"   Base: {BASE_DIR}")
print(f"   Data: {DATA_DIR}")
print(f"   Checkpoints: {CHECKPOINT_DIR}")
print(f"   Models: {MODEL_DIR}")

## Cell 4: Load Dataset & Configuration

In [ ]:
# Configuration
CONFIG = {
    'batch_size': 50,  # Process 50 samples at a time
    'max_retries': 3,  # Retry failed downloads
    'timeout': 30,     # Timeout per light curve download (seconds)
    'save_interval': 100,  # Save checkpoint every 100 samples
    'use_parallel': False,  # Parallel processing (set True if stable)
}

print("⚙️ Configuration:")
for key, val in CONFIG.items():
    print(f"   {key}: {val}")

# Load supervised dataset
supervised_path = DATA_DIR / 'supervised_dataset.csv'
if not supervised_path.exists():
    raise FileNotFoundError(f"❌ Dataset not found: {supervised_path}")

samples_df = pd.read_csv(supervised_path)

print(f"\n✅ Loaded: {supervised_path}")
print(f"   Total samples: {len(samples_df):,}")
print(f"   Positive: {samples_df['label'].sum():,} ({samples_df['label'].mean():.1%})")
print(f"   Negative: {(~samples_df['label'].astype(bool)).sum():,}")
print(f"   Columns: {list(samples_df.columns)}")

# Add sample_id and ensure required columns
if 'sample_id' not in samples_df.columns:
    samples_df['sample_id'] = [f"SAMPLE_{i:06d}" for i in range(len(samples_df))]

if 'tic_id' not in samples_df.columns:
    if 'tid' in samples_df.columns:
        samples_df['tic_id'] = samples_df['tid']
    elif 'target_id' in samples_df.columns:
        samples_df['tic_id'] = samples_df['target_id']

print(f"\n✅ Dataset prepared with {len(samples_df):,} samples")

## Cell 5: Feature Extraction Functions (with Checkpointing)

In [ ]:
def extract_features_from_lightcurve(
    time: np.ndarray,
    flux: np.ndarray,
    period: float,
    duration: float,
    depth: float
) -> Dict[str, float]:
    """Extract BLS features from light curve."""
    features = {}
    
    # Basic statistics
    features['flux_mean'] = np.nanmean(flux)
    features['flux_std'] = np.nanstd(flux)
    features['flux_median'] = np.nanmedian(flux)
    features['flux_mad'] = np.nanmedian(np.abs(flux - np.nanmedian(flux)))
    features['n_points'] = len(time)
    features['time_span'] = float(time[-1] - time[0])
    
    # Input parameters
    features['input_period'] = period
    features['input_duration'] = duration
    features['input_depth'] = depth
    
    # BLS analysis
    try:
        bls = BoxLeastSquares(time, flux)
        periods = np.linspace(0.5, 15.0, 2000)  # Reduced for speed
        durations = np.linspace(0.05, 0.5, 10)
        bls_result = bls.power(periods, durations)
        
        features['bls_power'] = float(bls_result.power.max())
        features['bls_period'] = float(bls_result.period[np.argmax(bls_result.power)])
        features['bls_duration'] = float(bls_result.duration[np.argmax(bls_result.power)])
        features['bls_depth'] = float(bls_result.depth[np.argmax(bls_result.power)])
        features['bls_snr'] = float(bls_result.depth_snr[np.argmax(bls_result.power)])
    except Exception as e:
        features.update({
            'bls_power': 0.0, 'bls_period': period, 'bls_duration': duration,
            'bls_depth': depth, 'bls_snr': 0.0
        })
    
    return features


def download_and_extract_single_sample(row: pd.Series, retries: int = 3) -> Optional[Dict]:
    """Download light curve and extract features for single sample."""
    for attempt in range(retries):
        try:
            tic_id = int(float(row['tic_id']))
            
            # Search for light curves (any sector)
            search_result = lk.search_lightcurve(f"TIC {tic_id}", author='SPOC')
            if search_result is None or len(search_result) == 0:
                return None
            
            # Download first available light curve
            lc_collection = search_result.download_all()
            if lc_collection is None or len(lc_collection) == 0:
                return None
            
            lc = lc_collection[0].remove_nans().normalize()
            
            # Extract features
            features = extract_features_from_lightcurve(
                lc.time.value, lc.flux.value,
                row.get('period', 1.0),
                row.get('duration', 0.1),
                row.get('depth', 0.01)
            )
            
            features['sample_id'] = row['sample_id']
            features['tic_id'] = tic_id
            features['label'] = row['label']
            
            return features
            
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
                continue
            return None
    
    return None


def load_checkpoint(checkpoint_path: Path) -> pd.DataFrame:
    """Load features from checkpoint."""
    if checkpoint_path.exists():
        df = pd.read_parquet(checkpoint_path)
        print(f"📂 Loaded checkpoint: {len(df)} samples")
        return df
    return pd.DataFrame()


def save_checkpoint(features_df: pd.DataFrame, checkpoint_path: Path):
    """Save features to checkpoint."""
    features_df.to_parquet(checkpoint_path, index=False)
    print(f"💾 Saved checkpoint: {len(features_df)} samples")


print("✅ Feature extraction functions defined")

## Cell 6: Extract Features with Checkpointing (THE BIG ONE)

In [ ]:
# Setup checkpoint
checkpoint_path = CHECKPOINT_DIR / 'features_checkpoint.parquet'
features_df = load_checkpoint(checkpoint_path)

# Determine which samples still need processing
if len(features_df) > 0:
    processed_ids = set(features_df['sample_id'].values)
    remaining_samples = samples_df[~samples_df['sample_id'].isin(processed_ids)]
else:
    remaining_samples = samples_df.copy()

print(f"📊 Progress Summary:")
print(f"   Total samples: {len(samples_df):,}")
print(f"   Already processed: {len(features_df):,}")
print(f"   Remaining: {len(remaining_samples):,}")
print()

if len(remaining_samples) == 0:
    print("✅ All samples already processed!")
else:
    print(f"🚀 Starting feature extraction for {len(remaining_samples):,} samples")
    print(f"   Estimated time: {len(remaining_samples) * 3 / 3600:.1f} hours (3 sec/sample)")
    print()
    
    start_time = time.time()
    new_features = []
    failed_count = 0
    
    # Process in batches
    for batch_start in range(0, len(remaining_samples), CONFIG['batch_size']):
        batch_end = min(batch_start + CONFIG['batch_size'], len(remaining_samples))
        batch = remaining_samples.iloc[batch_start:batch_end]
        
        print(f"\n📦 Batch {batch_start//CONFIG['batch_size'] + 1}: "
              f"Processing samples {batch_start+1}-{batch_end}")
        
        # Process batch with progress bar
        for _, row in tqdm(batch.iterrows(), total=len(batch), desc="  Processing"):
            result = download_and_extract_single_sample(row, CONFIG['max_retries'])
            
            if result is not None:
                new_features.append(result)
            else:
                failed_count += 1
            
            # Save checkpoint every N samples
            if len(new_features) % CONFIG['save_interval'] == 0 and len(new_features) > 0:
                temp_df = pd.concat([features_df, pd.DataFrame(new_features)], ignore_index=True)
                save_checkpoint(temp_df, checkpoint_path)
        
        # Progress stats
        elapsed = time.time() - start_time
        processed = len(new_features) + len(features_df)
        success_rate = (len(new_features) / (len(new_features) + failed_count)) * 100 if (len(new_features) + failed_count) > 0 else 0
        
        print(f"  ✅ Success: {len(new_features):,} | ❌ Failed: {failed_count:,} | Rate: {success_rate:.1f}%")
        print(f"  ⏱️ Elapsed: {elapsed/60:.1f} min | Remaining: {(len(remaining_samples)-batch_end)*elapsed/(batch_end)/60:.1f} min")
    
    # Final save
    if len(new_features) > 0:
        features_df = pd.concat([features_df, pd.DataFrame(new_features)], ignore_index=True)
        save_checkpoint(features_df, checkpoint_path)
    
    print(f"\n🎉 Feature extraction complete!")
    print(f"   Total samples processed: {len(features_df):,}")
    print(f"   Success rate: {len(features_df)/(len(features_df)+failed_count)*100:.1f}%")
    print(f"   Total time: {(time.time()-start_time)/3600:.2f} hours")

# Define feature columns
feature_cols = [col for col in features_df.columns if col not in ['sample_id', 'tic_id', 'label']]

print(f"\n✅ Feature columns defined: {len(feature_cols)} features")
print(f"   Features: {feature_cols[:5]}... (showing first 5)")

## Cell 7: Prepare Training Data

In [ ]:
# Prepare training data
X = features_df[feature_cols].values
y = features_df['label'].values
groups = features_df['sample_id'].apply(lambda x: hash(str(x)) % 10000).values

# Handle invalid values
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 Training data prepared:")
print(f"   Total samples: {len(X):,}")
print(f"   Features: {len(feature_cols)}")
print(f"   Train: {len(X_train):,} ({y_train.mean():.1%} positive)")
print(f"   Test: {len(X_test):,} ({y_test.mean():.1%} positive)")

## Cell 8: Train Model

In [ ]:
# GPU detection
def get_xgboost_gpu_params():
    try:
        import torch
        if torch.cuda.is_available():
            print("✅ GPU detected")
            return {'tree_method': 'hist', 'device': 'cuda'}
    except:
        pass
    print("ℹ️ Using CPU")
    return {'tree_method': 'hist'}

gpu_params = get_xgboost_gpu_params()

# Create pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', xgb.XGBClassifier(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.05,
        random_state=42,
        eval_metric='logloss',
        **gpu_params
    ))
])

print(f"\n🚀 Training model...")
start_time = time.time()

pipeline.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"✅ Training complete ({train_time:.1f} seconds)")

# Evaluate
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
y_pred = pipeline.predict(X_test)

ap_score = average_precision_score(y_test, y_pred_proba)
auc_score = roc_auc_score(y_test, y_pred_proba)

print(f"\n📊 Test Performance:")
print(f"   AUC-PR:  {ap_score:.4f}")
print(f"   AUC-ROC: {auc_score:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

## Cell 9: Save Model & Artifacts

In [ ]:
# Save model
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_path = MODEL_DIR / f'exoplanet_model_{timestamp}.pkl'
joblib.dump(pipeline, model_path)
print(f"✅ Model saved: {model_path}")

# Save feature columns
feature_path = MODEL_DIR / f'feature_columns_{timestamp}.json'
with open(feature_path, 'w') as f:
    json.dump(feature_cols, f, indent=2)
print(f"✅ Features saved: {feature_path}")

# Save metrics
metrics = {
    'timestamp': timestamp,
    'n_samples': len(X),
    'n_features': len(feature_cols),
    'train_samples': len(X_train),
    'test_samples': len(X_test),
    'train_time_seconds': train_time,
    'test_auc_pr': float(ap_score),
    'test_auc_roc': float(auc_score)
}

metrics_path = MODEL_DIR / f'metrics_{timestamp}.json'
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"✅ Metrics saved: {metrics_path}")

# Save features dataframe
features_path = MODEL_DIR / f'features_{timestamp}.parquet'
features_df.to_parquet(features_path, index=False)
print(f"✅ Features dataframe saved: {features_path}")

print(f"\n🎉 All artifacts saved to {MODEL_DIR}")

## Training Complete! 🎉

### What was saved:
1. **Model**: `exoplanet_model_{timestamp}.pkl`
2. **Features**: `feature_columns_{timestamp}.json`
3. **Metrics**: `metrics_{timestamp}.json`
4. **Features DataFrame**: `features_{timestamp}.parquet`
5. **Checkpoint**: `features_checkpoint.parquet` (for resuming)

### Next Steps:
- Use `04_newdata_inference.ipynb` for predictions
- If interrupted, just re-run Cell 6 - it will resume from checkpoint!

### Performance Tips:
- Colab Pro has longer timeouts
- Can pause and resume anytime
- Checkpoints saved to Google Drive